# AlgorithmicThinking: PeakFinding

<a href="https://www.youtube.com/watch?v=HtSuA80QTyo&list=PLUl4u3cNGP61Oq3tWYp6V_F-5jb5L2iHb&index=2" title="1. Algorithmic Thinking, Peak Finding"><img src="http://i3.ytimg.com/vi/HtSuA80QTyo/hqdefault.jpg" alt="1. Algorithmic Thinking, Peak Finding" /></a>


In [1]:
using BenchmarkTools

# 1-D Peak Finding

Given Array of 'n' numbers (1-indexed), a number nums[i] is defined as a peak if nums[i-1]=<nums[i]>=nums[i+1]

Can assume num[0]=num[n]=-inf

Return a peak if it exists

for eg: Position 2 is a peak if and only if b ≥ a and b ≥ c. Position 9 is a peak if i ≥ h.

`a,b,c,d,e,f,g,h,i`

`1,2,3,4,5,6,7,8,9`

Find peak if it exist.

1. Straight forward stepping algorithm that checks each number from left
    - Looks at n/2 elements on average, could look at n elements in the worst case $\Theta(n)$ => gives both lower and upper bound. O() is just upper bound. Algorithm complexity is linear 
    - How can we lower this complexity?

What if we start in the middle and look at n/2 elements

2. Recursive algorithm (divide & conquer)
    - if a[n/2] < a[n/2-1] then only look at left half, 
    - elseif a[n/2] < a[n/2+1]then look right hald
    - else n/2 position is a peak
    - work algorithm does on input of size n : $T(n)  = T(n/2) + \Theta(1)$ with base case $T(1) =\Theta(1)$
        - where, $\Theta(1)$ is time to compare a [n/2] neighbors
        - $T(n) = \underset{log_2 n \text{ times}}{\Theta(1) + \dots + \Theta(1)} = \Theta(log_2 n)$ 
    
    

In [2]:
"""Test if the index is the peak"""
function isPeak(arr, idx)
    idx == 1 && return arr[idx] >= arr[idx+1]
    idx == length(arr) && return arr[idx] >= arr[idx-1]
    return arr[idx-1] <= arr[idx] >= arr[idx+1]
end

arr = [1,2,3,4,7,3,5,2,6,7,9,7,2]
isPeak(arr, 1), isPeak(arr,5), isPeak(arr,13)

(false, true, false)

In [3]:
using Random
Random.seed!(0)
numArr = rand(-1000:1000, 10000);

In [4]:
"""Start at the left and look at all elements until the condition is fulfilled"""
function naivePeakSearch(arr)::Int32
    peaks = nothing
    for i in eachindex(arr)   # same as 1:length(arr)
        if isPeak(arr,i)
            return i
        end
    end
end

@benchmark naivePeakSearch(numArr)  setup=(numArr=rand(-10000:10000, 1_000_000))  # values in ns, should be higher

BenchmarkTools.Trial: 859 samples with 1000 evaluations.
 Range (min … max):  2.920 ns … 24.226 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     5.518 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.531 ns ±  2.550 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █     ▂       ▄           ▁                                 
  █▇▂▂▁▁█▅▁▂▁▁▁▁█▁▅▃▂▁▁▁▁▁▂▆█▄▄▂▂▁▂▁▁▁▃▂▁▂▂▂▄▁▁▁▁▂▂▁▁▂▁▁▁▁▁▂ ▃
  2.92 ns        Histogram: frequency by time        13.5 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In the worst caset, the peak might be at the right and we might have to look at all elements. The worse case complexity is $\Theta(n)$.

But generally arrays of numbers are generally are correlated, hence following the increasing or decreasing trend can be a good way to find peaks, it there's only one peak we can find it recursively as follows.

In [5]:
using Distributions
genRandPeak(n) = rand(TriangularDist(-n,n),n)
genRandPeak(5)'

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 3.95789  0.649545  -0.927318  0.148113  0.480864

In [6]:
"""Recursive Peak Search: Divide and Conquer
    Look at n/2, 
    If nums[n/2]<nums[n/2+1], look right and recurse, 
    If nums[n/2]<nums[n/2-1], look left and recurse
    return nums[n/2]
"""
function recursivePeakSearch(arr, lo, hi)::Int32
    mid = (lo + hi) ÷ 2
    arr[mid] < arr[mid-1] && return recursivePeakSearch(arr, lo, mid)
    arr[mid] < arr[mid+1] && return recursivePeakSearch(arr, mid, hi)
    return mid
end

@benchmark recursivePeakSearch(numArr, 0 , length(numArr)) setup=(numArr=genRandPeak(100_000))

BenchmarkTools.Trial: 2401 samples with 1000 evaluations.
 Range (min … max):  4.048 ns … 124.296 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.084 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.873 ns ±   7.377 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █   ▁▇▂  ▂▅▃   ▄▃▂  ▂▃▁  ▁▁    ▁                            ▁
  █▅▁▁███▆▁████▁▇███▇█████▆████▆██▆▇▆█▇▇▆▆▆█▃▄▆▆▆▅▁▁▆▇▆▅▁▁▄▃▅ █
  4.05 ns      Histogram: log(frequency) by time      34.8 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

## 2-D Peak Finding

Given a 2-D array Matrix, an element Matrix[i][j] is a hill (or peak) iff

- Matrix[i-1, j] <= Matrix[i, j] >= Matrix[i+1, j]
- Matrix[i, j-1] <= Matrix[i. j] >= Matrix[i, j+1]

In [7]:
arr2d = [1 2 3 4;
        14 13 12 10;
        15 9 11 18;
        16 17 19 20]

4×4 Matrix{Int64}:
  1   2   3   4
 14  13  12  10
 15   9  11  18
 16  17  19  20

In [8]:
"""Returns neighbours of a cell in a matrix"""
function neighbors1(arr, i, j)
    r, c = size(arr)
    neighbors = []
    for idx in [(i-1,j), (i+1,j),(i,j-1),(i,j+1)]
        if !(idx[1] in [0,r+1]) && !(idx[2] in [0,c+1])
            push!(neighbors, idx)
        end
    end
    return neighbors
end

neighbors1(arr2d,4,4)

2-element Vector{Any}:
 (3, 4)
 (4, 3)

In [9]:
"""Returns neighbours of a cell in a matrix"""
function neighbors(arr, idx)
    r, c = size(arr)    
    neighborIdx = [CartesianIndex(idx)] .+ CartesianIndex.([(1,0),(0,1),(-1,0),(0,-1)])
    validIdx = [checkbounds(Bool,arr,idx) for idx in neighborIdx]
    # return arr[neighborIdx[validIdx]]
    return neighborIdx[validIdx]
end

neighbors(arr2d, (2,2))

4-element Vector{CartesianIndex{2}}:
 CartesianIndex(3, 2)
 CartesianIndex(2, 3)
 CartesianIndex(1, 2)
 CartesianIndex(2, 1)

In [10]:
"""Greedy Ascent follows direction of largest increase to find a peak, has Θ(nm) complexity
Strategy: Pick arbitrary midpoint, return if peak else choose highest neighbor and repeat
"""
function greedyAscent(arr)
    idx = CartesianIndex(size(arr).÷2)  # start at midpoint
    while true
        neighborIdx = neighbors(arr,idx)
        all(arr[idx] .> arr[neighborIdx]) && return idx  # current index is peak
        idx = neighborIdx[argmax(arr[neighborIdx])]
    end
    
end
greedyAscent(arr2d)

CartesianIndex(4, 4)

In [11]:
Random.seed!(10)
mat = rand(0:50, 10,10)

10×10 Matrix{Int64}:
  0   7   4  34   9  12  14  29   6  10
  7  50  38  46   4  15  11  32   6  31
 47  49   9  18  21  36  39   6   0  44
 47  40  12  13  13  22  18  41  43  37
 42   0   6  44  37  48  11  16  35   3
 20  43   8  16  46  14   3  43  21  49
 10   3  50  39  13   3  49  48  37   7
  1  44  32   5  16   8  24  21   5  20
 41  50  24  28   7  34  12  39  38   0
 42  32  24  16  39  44   2  30  14  21

In [12]:
greedyAscent(mat) # There are other larger peaks as well, which this algo misses

CartesianIndex(5, 6)

In [13]:
""" Divide and Conquer
    1. Pick Middle column j =m/2 and find 1-D peak at column and then find 1D peak at row i. But 2D-peak may not exist on row i. as in arr2d below
    2. Divide and conquer with Global maximum
        1. Pick middle column j = m/2 
        2. Find global maximum on column j at (i,j)
        3. compare (i,j-1) (i,j), (i,j+1) and peak greater one. if (i,j) is greater, it is a 2D-peak
        4. solve the new problem with half number of columns
        5. When you have a single column find global maximum

Complexity 
T(n,m) = T(n,m/2) + Θ(n) (finding column max)

T(n,m) = log(m) Θ(n) there are log m of Θ(n) operations

T(n,m) = Θ(n log(m))
"""
function recursive2dpeak(arr, colLo, colHi)
    col = (colHi + colLo) ÷ 2
    row = argmax(arr[col, :])
    colLo == colHi && return row, col
    arr[row,col-1] > arr[row,col] && return recursive2dpeak(arr, colLo, col-1)
    arr[row,col+1] > arr[row,col] && return recursive2dpeak(arr, col+1, colHi)
    return row, col
end


recursive2dpeak(arr2d, 0, size(arr2d)[2])

(4, 4)

In [14]:
recursive2dpeak(mat, 0, size(mat)[2])

(6, 5)